# COVID2019
Justin Fackrell
Uses data from Johns Hopkins https://github.com/CSSEGISandData/COVID-19

In [2]:
import pandas as pd
from pathlib import Path



def get_data(file):

    # COVID-19 contains data pulled from https://github.com/CSSEGISandData/COVID-19
    _dir = Path(r"..\COVID-19\csse_covid_19_data\csse_covid_19_time_series")
    
    df = pd.read_csv(_dir / file)
    df.drop(['Lat', 'Long'], inplace=True, axis=1)
    df.set_index(['Province/State', 'Country/Region'], inplace=True)
    df = df.T
    return df

df_confirmed_raw = get_data("time_series_covid19_confirmed_global.csv")
df_deaths_raw = get_data("time_series_covid19_deaths_global.csv")


In [86]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots



def prepro(country2states, df, info):

    idx = pd.IndexSlice
    
    data = dict()
    diff_data = dict()

    for country in country2states:
        states = country2states[country]
        for state in states:
            _id = f"{country} {state if not pd.isnull(state) else ''}".strip()

            n = pd.Series(df.loc[:,idx[state, country]])
            d = n.diff()
            data[_id] = n
            diff_data[_id] = d
    
    df_diff = pd.DataFrame(data=diff_data)
    df = pd.DataFrame(data=data)

    return df, df_diff
    
    
def plot_confirmed_plus_deaths(df_confirmed, df_deaths, info, start_date=None):
    
    def _plot(df, start_date, info):
        labels = list()

        if start_date:
            mask = (df.index > start_date)
            df = df.loc[mask]
            
        for state in df.columns:
            plt.plot(df.index, df[state])
            labels.append(f"{state}")

        plt.legend(labels=labels)
        plt.title(f"{info}")

    plt.figure(figsize=(20,10))
    ax = plt.subplot(2,1,1)
    _plot(df_confirmed, start_date, 'confirmed')
    ax = plt.subplot(2,1,2)
    _plot(df_deaths, start_date, 'deaths')

    plt.xlabel('date')
    

    
def plot_confirmed_plus_deaths_fancy(df_confirmed, df_deaths, info, start_date=None):
    
    
    
    def _plot(fig, row, df, start_date, info):
        
        palate = px.colors.sequential.Jet
        labels = list()

        if start_date:
            mask = (df.index > start_date)
            df = df.loc[mask]
            
        for color, state in enumerate(df.columns):
            fig.add_trace(go.Scatter(x=df.index, y=df[state], mode='lines', name=state, 
                                     line=go.scatter.Line(color=palate[color])), row=row, col=1)
            
            #plt.plot(df.index, df[state])
            #labels.append(f"{state}")

#        plt.legend(labels=labels)
#        plt.title(f"{info}")

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    #fig = go.Figure()
    _plot(fig, 1, df_confirmed, start_date, 'confirmed')
    _plot(fig, 2, df_deaths, start_date, 'deaths')
    fig.show()


In [87]:
country2states = {'US': [None], 'China': ['Hubei'], 'Italy': [None], 'Spain': [None], 'Norway': [None], 'United Kingdom': [None]}

df_confirmed, df_confirmed_diff = prepro(country2states, df_confirmed_raw, 'confirmed')
df_deaths, df_deaths_diff = prepro(country2states, df_deaths_raw, 'deaths')

plot_confirmed_plus_deaths_fancy(df_confirmed_diff, df_deaths_diff, "diff", "2020-03-01")

In [ ]:
df_confirmed_deaths.tail()

In [85]:
px.colors.sequential.Jet

['rgb(0,0,131)',
 'rgb(0,60,170)',
 'rgb(5,255,255)',
 'rgb(255,255,0)',
 'rgb(250,0,0)',
 'rgb(128,0,0)']